<img src="images/ublogo.png"/>

### CSE610 - Bayesian Non-parametric Machine Learning

  - Lecture Notes
  - Instructor - Varun Chandola
  - Term - Fall 2020

### Objective
The objective of this notebook is to discuss methods for scaling GPs to large datasets.

<div class="alert alert-info">

**Note:** This material is based on Chapter 8 of the GPML book.

</div>

## Scaling issues with GPs
One of the key issues with GP based analysis that it typically scales as $\mathcal{O}(N^3)$ with the size of data. The memory requirement for storing the Gram matrix is $\mathcal{O}(N^2)$, which can pose a challenge for even moderately sized data sets. Contrast this with parametric models, which typically store a small sized model and can discard the training data.

Here we will look at several strategies to scale GPs for large datasets:

### Method 1 - Reduced-rank approximations of the Gram matrix

The key computational step in GP Regression (and somewhat similar step in GP Classification) is the inversion of the matrix $K + \sigma^2_nI$ or solving the linear system, $(K + \sigma^2_n){\bf v} = {\bf y}$.

\begin{eqnarray}
\bar{f}_* & = & {\bf k}_*^\top(K + \sigma_n^2I)^{-1}{\bf y}\\
var[f_*] & = & k({\bf x}_*,{\bf x}_*) - {\bf k}_*^\top(K + \sigma_n^2I)^{-1}{\bf k}_*
\end{eqnarray}

This is a $\mathcal{O}(N^3)$ step. In the GPML book, the algorithm uses *Cholesky Decomposition* for computing the inverse. For a positive definite matrix, the Cholesky decomposition factorizes the matrix as:
$$
A = LL^\top
$$
where $L$ is a $(N \times N)$ *lower triangular matrix*. Calculating the Cholesky decomposition is $\frac{N^3}{6}$.
> Question: Will $(K + \sigma_n^2I)$ always be p.d.?

The quantity $(K + \sigma_n^2I)^{-1}{\bf y}$ can now be calculated as $(L^\top)^{-1}L^{-1}{\bf y}$. Solving $L^{-1}{\bf y}$ can be done in $\frac{N^2}{4}$ steps. 

> Verify: Calculate the predictive variance in (2) above.

What if the matrix $K$ has a reduced rank?
$$
K = QQ^\top
$$
where $Q$ is a $(N \times q)$ matrix. By the *Matrix Inversion Lemma*, one can rewrite the inverse as:
$$
(K+ \sigma_n^2I)^{-1} = (QQ^\top + \sigma_n^2I)^{-1} = \sigma_n^{-2} I_N - \sigma_n^{-2}(\sigma_n^2I_q+Q^\top Q)^{-1}Q^\top
$$
> **The Matrix Inversion Lemma**:
$$
(Z + UWV^\top)^{-1} = Z^{-1} - Z^{-1}U(W^{-1} + V^\top Z^{-1}U)^{-1}V^\top Z^{-1}
$$

Note that the task in (5) is reduced to inverting a $q\times q$ matrix.

**But will $K$ have a reduced rank?**
If $K$ was derived from an explicit feature expansion, and if the expanded feature space, $p > N$, then, yes, $K$ will have a reduced rank. But for many kernel functions, $p = \infty$, so that might not be true.

But we can still consider a reduced-rank *approximation* of $K$. Remember that for a positive semi-definite symmetric matrix, the optimal reduced-rank approximation, w.r.t. the Frobenius norm is $U_q\Lambda_q U_q^\top$, where $\Lambda_q$ is the diagonal matrix of the leading $q$ eigenvalues of $K$ and $U_q$ is the matrix of the corresponding $q$ eigenvectors. 

But there is a catch! Eigenvector analysis itself is $\mathcal{O}(N^3)$.

### Nystrom Approximation
Consider a subset of the entire data set, denoted by $I$. Let the remaining set of examples be denoted by $R$. Let $I$ consist of $M$ examples, which means that $R$ will have $N - M$ examples. Thus $K$ can be partitioned as:
$$
K = \left[
\begin{array}{cc}
K_{MM} & K_{M(N - M)}\\
K_{(N - M)M} & K_{(N-M)(N-M)}
\end{array}
\right]
$$
Nystrom approximation states that one can use the eigendecomposition of the smaller matrix, $K_{MM}$ to approximate the eigenvectors and eigenvalues of the original matrix, $K$. In fact, if the eigenvectors and eigenvalues of $K_{MM}$ are denoted as $\{{\bf u}_i\}_{i=1}^M$ and $\{\lambda_i\}_{i=1}^M$, respectively, then, the $M$ eigenvectors/values of $K$ can be approximated as:
\begin{eqnarray}
\tilde{\lambda}_i^{(N)} & \triangleq & \frac{N}{M}\lambda_i^{(M)} & i = 1,\ldots,M\\
\tilde{{\bf u}}_i^{(N)} & \triangleq & \sqrt{\frac{M}{N}}\frac{1}{\lambda_i^{(M)}}K_{NM}{\bf u}_i^{(M)} & i = 1,\ldots,M\\
\end{eqnarray}

Now, remember that we can approximate $K$ using the first $M$ eigenvectors/values, i.e., $\tilde{K} = \sum_{i=1}^M \tilde{\lambda}_i^{(N)} \tilde{{\bf u}}_i^{(N)}(\tilde{{\bf u}}_i^{(N)})^\top$. Using (8) and (9), we get:
$$
\tilde{K} = K_{NM}K^{-1}_{MM}K_{MN}
$$
where $K_{NM} = K_{MN}^\top$.

This is known as the **Nystrom Approximation** of $K$. This matrix will have a reduced rank, and can make the GP analysis much more efficient. 

### Approximate methods based on greedy selection of an "active set"
We will discuss three methods here:
1. Subset of regressors (SR),
2. Subset of data points (SD), and,
3. Projected process (PP)

All three methods have a common first step - find a subset of training examples of size $M$ from the given training set of size $N$, where $M < N$. This is also known as the **active set** and is denoted by $I$. The set of remaining examples are denoted by $R$. 

Selecting $I$ can be done in different ways. Easiest way would be to select a random subset. But other "greedy" approaches exist as well and will be discussed later on.


### Subset of Regressors
Consider the predictive mean expression for GPR, $\bar{f}({\bf x}_*) = {\bf k}^\top({\bf x}_*)(K + \sigma_n^2I)^{-1}{\bf y}$, which can be written as $\bar{f}_* = \boldsymbol\alpha^\top{\bf k}({\bf x}_*) = \sum_{i=1}^N \alpha_ik({\bf x_i,x_*})$, where $\boldsymbol\alpha = (K + \sigma_n^2I)^{-1}{\bf y}$. 
> Here we are using a slightly different notation in ${\bf k}({\bf x}_*)$ which is a $N$ length vector consisting of kernel function evaluation between the test example, ${\bf x}_*$ and all training examples.

Estimating the "coefficient vector" $\boldsymbol\alpha$ is equivalent to solving a linear regression problem. In fact, it has been shown that GP regression is equivalent to a finite-dimensional linear regression model with a prior $\boldsymbol\alpha \sim \mathcal{N}({\bf 0},K^{-1})$. Even if we take a finite linear regression approach we will have to solve a $\mathcal{O}(N^3)$ problem.

However, if we solve the finite regression problem with fewer examples ($M < N$), one can come up with an approximate solution. 
$$
\bar{f}_* = \boldsymbol\alpha^\top{\bf k}_M({\bf x}_*)  = \sum_{i=1}^M \alpha_i k({\bf x}_*,{\bf x}_i)
$$
where $\boldsymbol\alpha_M \in \mathcal{N}({\bf 0},K^{-1}_{MM})$. The solution to this comes to:
\begin{eqnarray}
\bar{f}_* & = & {\bf k}_M({\bf x}_*)^\top(K_{MN}K_{NM} + \sigma^2_nK_{MM})^{-1}K_{MN}{\bf y}\\
var[f_*] & = & \sigma^2_n{\bf k}_M({\bf x}_*)^\top(K_{MN}K_{NM} + \sigma^2_nK_{MM})^{-1}{\bf k}_M({\bf x}_*)
\end{eqnarray}
> The above result can be obtained by applying kernel-regression type approach to the problem.

This is known as the **subset of regressors** (SR) approach.
> Question: What will be the complexity of the SR approach in terms of $M$ and $N$?

#### How to choose the subset $I$
- Randomly
- Run clustering on the original data set with $M$ clusters and choose the centroids
- Iterative (greedy) selection

### The Iterative (Greedy) Selection of subset
This can be used by any method that relies on a subset of examples. The general scheme is:
- Start with an empty active set $I$ and $R$ containing all of the samples
- At every step, choose one example from $R$ and move it to $I$ based on a greedy criterion
  * Often, a random subset of $R$ is chosen to select the next candidate

The greedy criterion is the key. One option that has been explored is to choose the example which gives the smallest residual sum of squares (RSS) - $\Vert {\bf y} - K_{NM}\boldsymbol\alpha_M\Vert_2^2$.

### Nystrom Method
We have already seen this before. The SR and Nystrom are very similar, however it is claimed that Nystrom method is not very good for smaller $M$.

### Subset of Datapoints
Perhaps the simplest approach is to just use a smaller subset of the training data for inference, also known as the **subset of datapoints** (SD) approach. This might appear to be too simplistic, since it involves discarding data. However, with a correct choice of the subset, one can get good approximations. 

### Other approaches
1. **Projected Process Approximation** (PP) - a combination of SD and SR methods
2. **Bayesian Committee Machines** (BCM) - splits data into $p$ parts and makes an assumption that:
$$
p({\bf y}_1,{\bf y}_2,\ldots,{\bf y}_p\vert f_*,X) \approx \prod_{i=1}^p p({\bf y}_i\vert  f_*,X_i)
$$
where $X_i$ and ${\bf y}_i$ are the inputs and targets for the examples in the $i^{th}$ partition.

## Comparing approximation methods

<img src="images/complexity_approx_gp.png"/>

<img src="images/comparisons_approx_gp.png"/>

## Approximate methods for classification

Very similar to GPR, except that we have to do the approximation on top of the approximation done for inference, i.e., Laplace approximation or Expectation Propagation.